#### Here will be covered : 
- Sentence Embedding
- Sentence Similarity
- Semantic Search
- Clustering

Resources:
- https://www.sbert.net/index.html
- https://www.sbert.net/docs/pretrained_models.html

In [1]:
%pip install -U sentence-transformers

# sentence-transformers (BERT) is a library for computing sentence embeddings (sentence vectors) in Python and PyTorch. 

Note: you may need to restart the kernel to use updated packages.


**Generate Embeding**

In [1]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
# all-MiniLM-L6-v2 is a transformer model trained on a large corpus of English sentences and their paraphrases. It is a smaller version of the MiniLM model (6 layers instead of 12 layers). It was trained on the AllNLI and STS benchmark datasets. And it is the best performing model on the STS benchmark dataset.

/home/mijanur/Desktop/AI/DL NLP/natural_language_processing/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
sentences = ['This framework generates embeddings for each input sentence','Sentences are passed as a list of string.', 'The quick brown fox jumps over the lazy dog.']

embeddings = model.encode(sentences, convert_to_tensor=True)

for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

In [5]:
text1 = """
Gradient descent is an optimization algorithm which is commonly-used to train machine learning models and neural networks. Training data helps these models learn over time, and the cost function within gradient descent specifically acts as a barometer, gauging its accuracy with each iteration of parameter updates.
"""
text2 = """
Gradient descent (GD) is an iterative first-order optimisation algorithm used to find a local minimum/maximum of a given function. This method is commonly used in machine learning (ML) and deep learning(DL) to minimise a cost/loss function (e.g. in a linear regression). Due to its importance and ease of implementation, this algorithm is usually taught at the beginning of almost all machine learning courses.
"""
text3 = """
Artificial intelligence is the simulation of human intelligence processes by machines, especially computer systems. Specific applications of AI include expert systems, natural language processing, speech recognition and machine vision.
"""

text4 = """
Natural language processing (NLP) is a branch of artificial intelligence (AI) that enables computers to comprehend, generate, and manipulate human language. Natural language processing has the ability to interrogate the data with natural language text or voice.
"""

text5 = """
Gradient Descent is known as one of the most commonly used optimization algorithms to train machine learning models by means of minimizing errors between actual and expected results. Further, gradient descent is also used to train Neural Networks.

In mathematical terminology, Optimization algorithm refers to the task of minimizing/maximizing an objective function f(x) parameterized by x. Similarly, in machine learning, optimization is the task of minimizing the cost function parameterized by the model's parameters. The main objective of gradient descent is to minimize the convex function using iteration of parameter updates. Once these machine learning models are optimized, these models can be used as powerful tools for Artificial Intelligence and various computer science applications.
"""

# This is the summerized version of text5
text6 = """
 Gradient Descent is an optimization algorithm used to train machine learning models by minimizing errors between actual and expected results. It is also used to train Neural Networks and minimize the cost function parameterized by the model's parameters.
"""

text7 = """
The Fear Nothing Blood Test is able to give you an accurate understanding of your health by checking several key health indicators. The standard Fear Nothing Blood Test can tell you about your: Vitamin levels. Hormone levels. Liver health.
"""

text8 = """
This subject only gives a brief description about different types of materials used in building construction for members like foundation, masonry, arches, lintels, balcony, roof, floor, doors, windows, stairs, plastering, painting and other general topics. Properties of various construction materials, their uses and different applications are discussed in this subject. 
"""

text9="""
Initial setting time for ideal cement mix is around 30 minutes for almost all kind of cements. For masonry cement it can be 90 minutes. Final setting time of ideal cement mix should be 10 hours at max. For masonry cement it shouldn’t exceed 24 hours.
"""

t1 = 'I like to eat apples.'
t2 = 'I like to play.'

# emd2 = model.encode("Tom deserves unbiased judgement")

In [9]:
len(text5.split())

114

In [6]:
emd1 = model.encode(t1)
emd2 = model.encode(t2)

**Cosine Similarity**

In [7]:
cos_similarity = util.cos_sim(emd1, emd2)
cos_similarity

tensor([[0.4809]])

**Compute Cosine Similarity Between all pairs**

In [ ]:
sentences = [
    'A man is eating food.',
    'A man is eating a piece of bread.',
    'A man is riding a horse.',
    'The girl is carrying a baby.',
    'A woman is playing violin.',
    'two men pushed carts through the woods.',
    'A man is riding a white horse on an enclosed ground.',
    'A monkey is playing drums.',
    'A cheetah is running behind its prey.'
]

In [ ]:
# Enode sentences to get their embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

# Compute cosine similarity between all pairs
cosine_scores = util.cos_sin(embeddings, embeddings)

In [ ]:
# add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        all_sentence_combinations.append([cosine_scores[i][j], i, j])

# Sort in decreasing order of the cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

# x[0] is the criteria for sorting

In [ ]:
# print the pairs according to their cosine similarity score
print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t\t {} \t\t {:.4f}".format(sentences[i], sentences[j], cosine_scores[i][j]))

In [ ]:
#### Using Hugging Face Transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


#### Semantic Search

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('clips/mfaq')

In [ ]:
question = "<Q>How many models can I host on Hugging Face?</Q>"
answer_1 = "<A>All plans come with unlimited private models and datasets."
answer_2 = "<A>AutoNLP is an automatic way to train and deploy state-of-the-art NLP models, seamlessly integrated with the Hugging Face ecosystem."
answer_3 = "<A>Based on how much training data and model variants are created, we send you a compute cost and payment link - as low as $10 per job."

In [ ]:
query_embedding = model.encode(question)
corpus_embeddings = model.encode([answer_1, answer_2, answer_3])

results = util.semantic_search(query_embedding, corpus_embeddings)

#### Question Answering

Basically for question answering we have to perfrom also the previous tasks
Sequence is : 
1. First find the semantic similarity by ranking existing document
2. Then Semantic search
3. Find the top document -> Finnaly the QA step

In [ ]:
from transformers import pipeline

In [ ]:
qa_model = pipeline("question-answering")
question = "How many models can I host on HuggingFace?"
context = "All plans come with unlimited private models and datasets."
qa_model(question = question, context = context)

#### Clustering

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'Horse is eating grass.',
          'A man is eating pasta.',
          'A Woman is eating Biryani.',
          'The girl is carrying a baby.',
          'The baby is carried by the woman',
          'A man is riding a horse.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.',
          'A cheetah is running behind its prey.',
          'A cheetah chases prey on across a field.',
          'The cheetah is chasing a man who is riding the horse.',
          'man and women with their baby are watching cheetah in zoo'
          ]
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

In [ ]:
corpus_embeddings[0]

In [ ]:
# source: https://stackoverflow.com/questions/55619176/how-to-cluster-similar-sentences-using-bert

clustering_model = KMeans(n_clusters=4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)

In [ ]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])
clustered_sentences

In [ ]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])
clustered_sentences